Important Note: This code will not replicate the disease progression model used in 'Discovery of Parkinson’s disease states using a machine learning approach'. If you would like to use the specific model from the paper, please reach out to the authors.

In [ ]:
import numpy as np
import pandas as pd
import pickle 
import torch
import sys

### place path to Disease Progression Model repo here
sys.path.append('path-to-repo')

from piohmm import HMM

In [ ]:
# import data, note that this result is generated by 'LD_Data_Processing.ipynb'
with open('data_for_PIOHMM.pkl', 'rb') as handle:
    data = pickle.load(handle)

In [ ]:
x_train = data['x_train']
d_train = data['train_med']

In [ ]:
# don't include samples which only have one measurement, i.e. aren't time series
remove_idx = np.where(np.sum(~np.isnan(x_train[:,:,0]), axis=1) == 1)
x_train = np.delete(x_train, remove_idx, 0)
d_train = np.delete(d_train, remove_idx, 0)

In [ ]:
# set any LEDD values greater than 5000 to 620 and rescale
d_train[d_train > 5000] = 620
d_train[np.isnan(d_train)] =  0
d_train = d_train/np.max(d_train)

In [ ]:
# get time and observation masks
n, t, d, = x_train.shape

time_mask = np.ones((n,t))
for i in range(n):
    ind = np.where(~np.isnan(x_train[i, :, 0]))[0][-1] + 1
    time_mask[i, ind:] = 0
    
missing_mask = (~np.isnan(x_train[:, :, 0])).astype(float)

x_train[np.isnan(x_train)] = 0

In [ ]:
# convert everything to tensors
X = torch.Tensor(x_train).float()
D = torch.Tensor(d_train).float()
TM = torch.Tensor(time_mask).float()
OM = torch.Tensor(missing_mask).float()

In [ ]:
k = 8

In [ ]:
model = HMM(X, ins=D, k=k, TM=TM, OM=OM, full_cov=True, priorV=True, io=True, personalized=False,
            personalized_io=True, state_io=True, UT=True, device='cpu', eps=1e-18)

In [ ]:
params_hat, e_out, ll, elbo, mu_hat, L_hat = model.learn_model(num_iter=20000, intermediate_save=False)

PIOHMM_model = {'params': params_hat, 'e_out': e_out, 'Mi': mu_hat, 'Li': L_hat, 'elbo': elbo, 'll_seq': ll}

In [ ]:
# calculate the most probable sequence
mps = model.predict_sequence(params_hat, mu_hat)

In [ ]:
x_test = data['x_test']
d_test = data['test_med']

In [ ]:
# don't include samples which only have one measurement, i.e. aren't time series
remove_idx = np.where(np.sum(~np.isnan(x_test[:,:,0]), axis=1) == 1)
x_test = np.delete(x_test, remove_idx, 0)
d_test = np.delete(d_test, remove_idx, 0)

In [ ]:
# set any LEDD values greater than 5000 to 620 and rescale
d_test[d_test > 5000] = 620
d_test[np.isnan(d_test)] =  0
d_test = d_test/np.max(d_train)

In [ ]:
# get time and observation masks
n, t, d, = x_test.shape

time_mask = np.ones((n,t))
for i in range(n):
    ind = np.where(~np.isnan(x_test[i, :, 0]))[0][-1] + 1
    time_mask[i, ind:] = 0
    
missing_mask = (~np.isnan(x_test[:, :, 0])).astype(float)

x_test[np.isnan(x_test)] = 0

In [ ]:
# convert everything to tensors
X_test = torch.Tensor(x_test).float()
D_test = torch.Tensor(d_test).float()
TM_test = torch.Tensor(time_mask).float()
OM_test = torch.Tensor(missing_mask).float()

In [ ]:
# 'swap-in' the test data
model.change_data(X_test, ins=D_test, TM=TM_test, OM=OM_test, reset_VI=True, params=params_hat)

params_hat, e_out_test, ll_test, elbo_test, mu_hat_test, L_hat_test = model.learn_vi_params(params_hat, num_iter=15000)

In [ ]:
mps_test = model.predict_sequence(params_hat, mu_hat_test)

In [ ]:
PIOHMM_model['mu_hat_test'] = mu_hat_test
PIOHMM_model['L_hat_test'] = L_hat_test
PIOHMM_model['elbo_test'] = elbo_test

In [ ]:
with open('piohmm_result.pkl', 'wb') as handle:
    pickle.dump(PIOHMM_model, handle)